<a href="https://colab.research.google.com/github/jeaneigsi/news-detection/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fist of Judgement**


In [ ]:
!pip install transformers

In [ ]:
!unzip Faker.zip


Archive:  Faker.zip
  inflating: Fake.csv/Fake.csv       
  inflating: True.csv/True.csv       
replace il_1140xN.1287758547_2nwc.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 



```
# Ce texte est au format codefrom
tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.ops.numpy_ops import np_utils
from transformers import BertModel, TFBertModel
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
```



In [ ]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Input
from transformers import BertModel,TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.ops.numpy_ops import np_utils
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



```
# Ce texte est au format code
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
import numpy as np
from PIL import Image
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from transformers import AutoTokenizer
```



In [ ]:
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
import numpy as np
from PIL import Image
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import AutoTokenizer

In [ ]:
fake_data=pd.read_csv("/content/Fake.csv/Fake.csv")
real_data=pd.read_csv("/content/True.csv/True.csv")

In [ ]:
fake_data.shape
fake_data.head()


In [ ]:
real_data.shape
real_data.head()


In [ ]:
real_data['subject'].value_counts().plot(kind='barh')
rcParams['figure.figsize']=5,5
real_data['subject'].unique()

In [ ]:
fake_data['subject'].value_counts().plot(kind='barh')
rcParams['figure.figsize']=5,5
fake_data['subject'].unique()

In [ ]:
# wrdcld_mask=np.array("/content/kitten.png")


In [ ]:
wrdcld_mask=np.array(Image.open("/content/il_1140xN.1287758547_2nwc.png"))


In [ ]:
wrdcld_text=" ".join(real_data['text'].tolist())
print(wrdcld_text[:15])
print(len(wrdcld_text))

In [ ]:
wordcloud = WordCloud(width=1920, height=1080, mask=wrdcld_mask).generate(wrdcld_text)
fig=plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
fake_text=" ".join(fake_data["text"].tolist())


In [ ]:
wordcloud=WordCloud(width=1920,height=1080,mask=wrdcld_mask).generate(fake_text)
fig=plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
fake_data["label"]="fake"
real_data["label"]="real"

In [ ]:
fake_data.head()

In [ ]:
real_data.head()

In [ ]:
final_data=pd.concat([fake_data,real_data])
final_data.head(10)


In [ ]:
final_data.shape

In [ ]:
final_data=final_data.sample(frac=1).reset_index(drop=True)

In [ ]:
final_data.head(10)

In [ ]:
#connaitres les valeurs nulles par colones

final_data.isnull().sum()

In [ ]:
final_data.drop(['subject','date'],axis=1)

In [ ]:
final_data.head()


In [ ]:
final_data.label.value_counts()

In [ ]:
sns.set_theme(style="whitegrid")
sns.countplot(x=final_data["label"])

In [ ]:
final_data.head(5)

In [ ]:
final_data["text"]=final_data["title"] + final_data["text"]

final_data.head()

In [ ]:
final_data=final_data[["text","label"]]
#crrer un nouveau tableau
final_data.head()

In [ ]:
final_data['label']=final_data['label'].map({'real':1, 'fake':0})

In [ ]:
final_data.head()

In [ ]:
final_data.label[500]

In [ ]:
final_data.text[500]

In [ ]:
data_text=final_data["text"]
data_label=final_data["label"]

In [ ]:
data_text.head()

In [ ]:
bert_tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
bert_model=TFBertModel.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(data_text, data_label,stratify=data_label, test_size=0.3,random_state=42)

In [ ]:
def tokenize(X):

    X = bert_tokenizer(
        text = list(X),
        add_special_tokens = True,
        max_length = 100,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True
        )
    return X

In [ ]:
X_train_token = tokenize(X_train)


In [ ]:
X_test_token = tokenize(X_test)

In [ ]:
maxlen=100

In [ ]:
def create_model():
    dropout_rate=0.2
    input_ids=Input(shape=(maxlen,),dtype=tf.int32)
    input_mask=Input(shape=(maxlen,),dtype=tf.int32)
    bert_layer=bert_model([input_ids,input_mask])[1]
    x=Dropout(0.5)(bert_layer)
    x=Dense(64,activation="tanh")(x)
    x=Dropout(0.2)(x)
    x=Dense(1,activation="sigmoid")(x)
    model = Model(inputs=[input_ids, input_mask], outputs=x)
    return model


In [ ]:
model=create_model()
model.summary()

In [ ]:
optimizer = Adam(learning_rate=1e-05, epsilon=1e-08, weight_decay=0.01,clipnorm=1.0)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = 'accuracy')

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max', verbose=1, patience=50,baseline=0.4,min_delta=0.0001,restore_best_weights=False)

In [ ]:
history = model.fit(x = {'input_1':X_train_token['input_ids'],'input_2':X_train_token['attention_mask']}, y = Y_train, epochs=10, validation_split = 0.2, batch_size = 30, callbacks=[callback])

In [ ]:
model.save('Tokenizer.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pred = np.where(model.predict({ 'input_1' : X_test_token['input_ids'] , 'input_2' : X_test_token['attention_mask']}) >=0.5,1,0)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
conf_matrix = confusion_matrix(Y_test,y_pred)
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix, figsize=(6, 6), cmap=plt.cm.Greens)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
print(classification_report(Y_test,y_pred))


**# Testons sur un texte**

In [ ]:
test_text="Cop Shares Racist Meme About Michelle Obama; Now That Cop Is Having A VERY Bad Day (IMAGES)After the election of Donald Trump many folks seem to see it as a permission slip to be as racist and vile as possible. However, here s the thing, you re still going to get called out as racist and vile. And one Alabama police officer just found this out the hard way.According to the Washington Post: Talladega Police Officer Joel Husk was terminated Wednesday for violating the department s social media and code of conduct policies, City Manager Patrick Bryant said. What did he do? So glad you asked: Husk had posted several memes on his Facebook page, including one showing Obama and Melania Trump.  Fluent in Slovenian, English, French, Serbian, and German,  it said over Trump s photo. Over Obama s, it read:  Fluent in Ghetto. Not only that, he posted several extraordinarily racist memes:via Washington Postvia Washington PostAccording to the City Manager, the statements were  deemed to be biased or racially insensitive or derogatory  and because of that, they  have to take action to correct it. If you re going to be a police officer and serve all the public, you can t assume black people standing up for their rights are equivalent to the KKK. That s about the most horrific equivalence imaginable.Also, according to WaPo: Husk, 37, who had been with the department for about two and a half years, had also shared a meme showing President Obama with the words:  Was Dallas a terrorist attack? Yes! Carried out by Obama s own homegrown terrorist group! Which is a blatant lie and anyone who were to feel that way belongs nowhere near law enforcement. The city took the proper action letting this racist cop go, and hopefully it will be an example to police departments all over the country that this sort of behavior simply cannot be tolerated.Trump s election must not be allowed to serve as a permission slip to bigots everywhere that it s fine to be as awful as possible, because here in the land of the free and the home of the brave, everyone is protected. Everyone, regardless of color, class, gender, sexual orientation, or creed.Featured Photo by Chip Somodevilla/Getty Images'"

In [ ]:
test_token = tokenize(test_text)


In [ ]:
test_text_pred = np.where(model.predict({ 'input_1' : test_token['input_ids'] , 'input_2' : test_token['attention_mask']}) >=0.5,1,0)


In [ ]:
test_text_pred

In [ ]:
if(test_text_pred[0]==0):
    print("News is Fake")
else:
    print("News is Real")